<a href="https://colab.research.google.com/github/lawrenceN/Effects_of_semantic_density_on_sentiment/blob/master/U4_M8_Creating_the_initial_K_Means_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OK, let's get to our example.  We have an array of centroids, or proposed centers for our clusters, and we have a group of points that we want to assign to those centroids. To do that, we're going to look at each point and find the distance to each centroid, then assign it to the closest one.

The first thing we need to do is expand the points and the centroids arrays so we can make the math work:

In [0]:
%matplotlib notebook

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100

field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()

graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):
        centroids_plot.set_data(
           centroids[:, 0], 
           centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

All right let's talk about what we're going to do with these things.  To assign each point to the closest centroid, we're going to find the distance between each point and each centroid, and assign the point to the centroid for which the distance is the smallest.

Now, remember back when we were talking about scalars and vectors, and how a vector has magnitude, or length, and also direction?  Well, we need to take that into consideration when we are trying to find the distance between two points.  

We'll start out by finding the displacement, which is basically the vector that goes from the tail of one to the head of the other, like this:

![image](https://s3-us-west-2.amazonaws.com/livevideo-collab-resources/Machine+learning+for+mere+mortals/figure4_8_1.png)

Fortunately, TensorFlow can easily do all of this with the subtract function:

In [0]:
%matplotlib inline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100

field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):

        displacement = tf.subtract(points_expanded, centroids_expanded)
        print(session.run(displacement))

        centroids_plot.set_data(
           centroids[:, 0], 
           centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

Since we're subtracting one set of tensors from another set of tensors, we'll wind up with a third set of tensors, as you can see. So each of those pairs represents the displacement, that is, the vector between the point and the centroid.  So the next thing we need to do is find the smallest vector for each point, because that's the closest centroid.  Remember...

![triangle_graphic](https://s3-us-west-2.amazonaws.com/livevideo-collab-resources/Machine+learning+for+mere+mortals/figure4_8_2.png)

`c**2 = a**2 + b**2`

So we could do the math to find the length of each of those 600 vectors, but we don't actually need to go that far. Instead of finding 

`min(sqrt(a**2 + b**2))`

We can just find

`min(a**2 + b**2)`

Because that's going to be the same, and it's going to save us 600 square root operations.  So let's go ahead and do that.

In [0]:
%matplotlib notebook

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from matplotlib import animation
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100

field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)
print(points_values)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):

        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)
        print(session.run(distances))

        centroids_plot.set_data(
               centroids[:, 0], 
               centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

So first we're squaring the values in the displacement matrix, which gives us





In [0]:
`[a**2, b**2]`

But then we need to add them up, and the way that we do that is with the reduce_sum() function.  What that does is it adds up the values on the specified dimension.  Here's what I mean by that. Say we have this tensor:

In [0]:
start = tf.constant([[[1,1,1], [1,1,1], [1,1,1], [1,1,1]], [[2,2,2], [2,2,2], [2,2,2], [2,2,2]]])

If we use tf.reduce_sum() without specifying a dimension, we wind up adding up all of the values for a single scalar:

In [0]:
print(tf.reduce_sum(start))
print(session.run(tf.reduce_sum(start)))

If we use `tf.reduce_sum(start, 0)` to add up the 0th level of groups, we have

In [0]:
print(tf.reduce_sum(start, 0))
print(session.run(tf.reduce_sum(start, 0)))

Meanwhile, `reduce_sum(start, 1)` adds up the second level of groups, so 

In [0]:
print(tf.reduce_sum(start, 1))
print(session.run(tf.reduce_sum(start, 1)))

reduce_sum(start, 2) adds up the third level...

In [0]:
print(tf.reduce_sum(start, 2))
print(session.run(tf.reduce_sum(start, 2)))

... and so on.

In the case of our distances, it's that third level that we want.  The shape starts as

`[3, 600, 2]`

and we're trying to get to 

`[3, 600]`

because that gives us the distance between each centroid and each point. So it's that 2, in the third position, that we want to reduce, so we're using

`distances = tf.reduce_sum(squared_displacement, 2)`

The next step is to assign each point to the centroid for which the distance is the smallest.  We'll do that using the `argmin()` function. The `argmin()` function sounds kind of weird, but what it's doing is giving you the index, or argument, of the smallest item in the specified axis, which in this case is the zeroth axis.

We start out with three distances for each point, and for each point, argmin returns the index of the lowest distance.  So when we run it, we wind up with an array of assignments.

In [0]:
%matplotlib inline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100

field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):

        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)

        assignments = tf.argmin(distances, 0) 
        assignment_values = session.run(assignments)
        print(assignment_values)
        
        centroids_plot.set_data(
               centroids[:, 0], 
               centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

So we're almost there with the math.  Now we need to create three partitions, or groups -- one for each centroid -- and assign each point to the one that corresponds to that minimum.

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100

field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):

        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)

        assignments = tf.argmin(distances, 0)
        assignment_values = session.run(assignments)

        partitions = tf.dynamic_partition(points,       
                                     tf.to_int32(assignment_values), 
                                     num_clusters)
        print(session.run(partitions))

        centroids_plot.set_data(
               centroids[:, 0], 
               centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

The `dynamic_partition()` function takes the set of points, or the data, and the assignments, as well as the number of partitions that we have, and creates those groupings for us.

Once we have that, we can go ahead and determine what the new centroids are by finding the average position of each grouping.

In [0]:
%matplotlib notebook

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100
field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)
print(points_values)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):
        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)
        
        assignments = tf.argmin(distances, 0)
        assignment_values = session.run(assignments)

        partitions = tf.dynamic_partition(points,       
                                      tf.to_int32(assignment_values), 
                                      num_clusters)

        newcentroids = np.zeros((num_clusters, 2))
        for i in range(num_clusters):
            newcentroids[i] = session.run(tf.reduce_mean(partitions[i], 0))
            
        centroids_plot.set_data(
               centroids[:, 0], 
               centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

First we'll create a new array called `newcentroids` to hold the `x` and `y` coordinates of each cluster.  Then for each cluster, we'll get the mean, or the average, of the `x` and `y` values for all the points in the cluster.

So as you can see here, we have new values for the centroids.

Now the key here is that as long as the centroids are moving, it means that the algorithm is still working on finding the optimal position for them. Once we get there, we can stop, so before we move on we'll see if we're ready.

First we're comparing the old centroid array with the new array. If they're equal, then the centroids haven't moved, because none of the points have switched centroids.  So our results aren't going to get any better, and we can stop trying.  So we'll set `not_finished` to `False` and output the number of steps it took and the centroids.

On other hand, if they're not equal, that means that we're still working on getting a better result, so we'll set the values of the `centroids` object to the new values, then expand the dimensions the way we did before so we're ready to start a new iteration.

Now we can run this right now, and we'll see the centroids move:

In [0]:
%matplotlib notebook

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100
field_size = 10
not_finished = True

centroids = np.random.random_sample([num_clusters, 2])

centroids = field_size * centroids
fig = plt.figure()

graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):
        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)
        assignments = tf.argmin(distances, 0)
        
        assignment_values = session.run(assignments)
        partitions = tf.dynamic_partition(points,       
                                      tf.to_int32(assignment_values), 
                                      num_clusters)

        newcentroids = np.zeros((num_clusters, 2))
        for i in range(num_clusters):
            newcentroids[i] = session.run(tf.reduce_mean(partitions[i], 0))

        if (np.array_equal(centroids, newcentroids)):
            not_finished = False
            print("Finished after "+str(this_iteration)+" steps.")
            print(centroids)
        else:
            centroids = newcentroids
            centroids_expanded = tf.expand_dims(centroids, 1)

        centroids_plot.set_data(
               centroids[:, 0], 
               centroids[:, 1])

    return centroids_plot, 

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=100)

plt.show()

There you can see that it stops after some number of steps, because it's not getting any better.

(Note that we specified `frames=200`, which means that the animation will iterate through the integers 0 through 200, then stop, even if we don't find the solution.)

The problem here, though, is that we still don't know which points belong to which cluster.  So let's take care of that next. 

First we need to define an object to hold the points to be plotted.  We'll do that the same way we created one to hold the centroids to plot:

In [0]:
points_plot = {}
points_plot[0], = graph.plot([], [], 'r.')
points_plot[1], = graph.plot([], [], 'b.')
points_plot[2], = graph.plot([], [], 'g.')

So we're creating the points_plot object.  What we want is for each cluster of points to have its own color, so we've got an array of points_plot objects, each of which, like the centroids_plot object, creates an empty data set and defines what it should look like. 

So the first group is going to be red dots, the second will be blue dots, and the third will be green dots.  Now, I'm going to confess that I'm taking a little shortcut here, and directly defining three sets because I know that we have 3, but if this were a real application I'd be defining arrays of colors, then just creating the number of clusters that we have, and so on, but I'll leave that as an exercise for the reader, as they say.

OK, now let's put them to use.

So if we're still working on getting better results, we'll start by clearing out the three groups of points to plot.  Then for each point, our goal is to assign the `x` and `y` coordinate to the proper `points_plot` grouping.

So we'll start out by getting a reference to the right grouping.  Remember, `assignment_values` is a list of cluster numbers, with one corresponding to each point.  Also, these are tuples, so don't forget the comma after both sides.

Next we're going to get all of the existing `x` values for that cluster, and we're going to append the `x` value for the current point to it.  We'll do the same thing for the `y` data.  Then we'll set the data for the cluster to be the new sets of `x` and `y` data.

Finally, we'll add the three sets of points to plot to the output of the function so that when the animation runs (you remember the animation, right? This is a discussion about the animation) the points will get plotted too.

So let's run it.

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from matplotlib import animation
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

num_points = 600
num_clusters = 3
num_iterations = 100
field_size = 10

not_finished = True
centroids = np.random.random_sample([num_clusters, 2])
centroids = field_size * centroids

fig = plt.figure()
graph = plt.axes(xlim=(-1*field_size, field_size), 
                 ylim=(-1*field_size, field_size))

centroids_plot, = graph.plot([], [], 'kx', markersize=15, markeredgewidth=3)

points_plot = {}
points_plot[0], = graph.plot([], [], 'r.')
points_plot[1], = graph.plot([], [], 'b.')
points_plot[2], = graph.plot([], [], 'g.')

points_build = np.random.laplace(field_size/4, field_size/4, [num_points,2])
points = tf.constant(points_build, dtype=tf.float64)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

points_values = session.run(points)

points_expanded = tf.expand_dims(points, 0)
centroids_expanded = tf.expand_dims(centroids, 1)

def updatefigure(this_iteration):

    global centroids, not_finished, num_iterations, points_expanded, centroids_expanded

    if (not_finished and this_iteration < num_iterations):
        displacement = tf.subtract(points_expanded, centroids_expanded)
        squared_displacement = tf.square(displacement)
        distances = tf.reduce_sum(squared_displacement, 2)
        
        assignments = tf.argmin(distances, 0)
        assignment_values = session.run(assignments)

        partitions = tf.dynamic_partition(points,       
                                      tf.to_int32(assignment_values), 
                                      num_clusters)

        newcentroids = np.zeros((num_clusters, 2))

        for i in range(num_clusters):
            newcentroids[i] = session.run(tf.reduce_mean(partitions[i], 0))

        if (np.array_equal(centroids, newcentroids)):
            not_finished = False
            print("Finished after "+str(this_iteration)+" steps.")
            print(centroids)
        else:
            centroids = newcentroids
            centroids_expanded = tf.expand_dims(centroids, 1)

            points_plot[0].set_data([], [])
            points_plot[1].set_data([], [])
            points_plot[2].set_data([], [])

            for i in range(num_points-1):
                this_set, = points_plot[assignment_values[i]],
                xdata = this_set.get_data()[0]
                xdata = np.append(xdata, points_values[i][0])
                ydata = this_set.get_data()[1]
                ydata = np.append(ydata, points_values[i][1])
                this_set.set_data(xdata, ydata)

            centroids_plot.set_data(
                            centroids[:, 0], 
                            centroids[:, 1])

    return centroids_plot, points_plot[0], points_plot[1], points_plot[2],

anim = animation.FuncAnimation(fig=fig, func=updatefigure, frames=200, interval=500)

plt.show()

OK!  Now we're getting somewhere!  We got TensorFlow to do k-means clustering for us, and we can see the results.

Just a quick warning, here, this can get kind of hypnotic, so try not to … get … sucked … in…

Right.  So next let's look at using clustering in a completely different way.  Let's look at clustering twitter topics.